# Lesson 2 - Retrieval Augmented Generation (RAG)

### Import the required packages

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
from datasets import load_dataset
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from tqdm.auto import tqdm

import ast
import os
import pandas as pd

In [3]:
from DLAIUtils import Utils

### Setup Pinecone

In [4]:
utils = Utils()
PINECONE_API_KEY = utils.get_pinecone_api_key()

In [ ]:
pinecone = Pinecone(api_key=PINECONE_API_KEY)
INDEX_NAME = utils.create_dlai_index_name(index_name="dl-ai")

if INDEX_NAME in [index.name for index in pinecone.list_indexes()]:
    pinecone.delete_index(name=INDEX_NAME)

print(INDEX_NAME)

pinecone.create_index(
    name=INDEX_NAME,
    dimension=1536,
    spec=ServerlessSpec(cloud="aws", region="us-west-2"),
    metric="cosine"
)

index = pinecone.Index(name=INDEX_NAME)
print(index)

### Load the Dataset

**Note:** To access the dataset outside of this course, just copy the following two lines of code and run it (remember to uncomment them first before executing):

In [14]:
!mkdir -p ./data && wget -q -O ./data/lesson2-wiki.csv.zip "https://www.dropbox.com/scl/fi/yxzmsrv2sgl249zcspeqb/lesson2-wiki.csv.zip?rlkey=paehnoxjl3s5x53d1bedt4pmc&dl=0"

In [16]:
!unzip ./data/lesson2-wiki.csv.zip -d ./data/

Archive:  ./data/lesson2-wiki.csv.zip
  inflating: ./data/wiki.csv         


In [17]:
max_articles_num = 500
df = pd.read_csv("./data/wiki.csv", nrows=max_articles_num)
df.head()

,id,metadata,values
1,1-0,"{'chunk': 0, 'source': 'https://simple.wikiped...","[-0.011254455894231796, -0.01698738895356655, ..."
2,1-1,"{'chunk': 1, 'source': 'https://simple.wikiped...","[-0.0015197008615359664, -0.007858820259571075..."
3,1-2,"{'chunk': 2, 'source': 'https://simple.wikiped...","[-0.009930099360644817, -0.012211072258651257,..."
4,1-3,"{'chunk': 3, 'source': 'https://simple.wikiped...","[-0.011600767262279987, -0.012608098797500134,..."
5,1-4,"{'chunk': 4, 'source': 'https://simple.wikiped...","[-0.026462381705641747, -0.016362832859158516,..."


### Prepare Embeddings and Upsert to Pinecone

In [19]:
prepped = []

for i, row in tqdm(df.iterrows(), total=df.shape[0]):

    prepped.append({
        "id": row["id"],
        "values": ast.literal_eval(row["values"]),
        "metadata": ast.literal_eval(row["metadata"])
    })

    if len(prepped) >= 250:
        index.upsert(prepped)
        prepped = []

100%|██████████| 500/500 [00:12<00:00, 40.61it/s] 


In [20]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 500}},
 'total_vector_count': 500}

### Connect to OpenAI

In [21]:
OPENAI_API_KEY = utils.get_openai_api_key()
openai_client = OpenAI(api_key=OPENAI_API_KEY)

In [22]:
def get_embeddings(articles, model="text-embedding-ada-002"):
    return openai_client.embeddings.create(input=articles, model=model)

### Run Your Query

In [24]:
query = "what is the berlin wall?"

embed = get_embeddings(articles=[query])
res = index.query(vector=embed.data[0].embedding, top_k=3, include_metadata=True)
text = [r["metadata"]["text"] for r in res["matches"]]
print("\n".join(text))

August 13  1961: Building of the Berlin Wall begins.
 August 14  1945: Japan announces its surrender at the end of World War II.
 August 14/15  1947: India is partitioned at independence from the UK, as the new mainly Islamic state of Pakistan is created.
 August 15  1960: The Republic of the Congo becomes independent.
 August 15  1971: Bahrain becomes independent.
 August 16  1977: Elvis Presley dies aged 42, leading to a worldwide outpouring of grief.
 August 17  1945: Indonesia declares independence from the Netherlands.
 August 17  1960: Gabon becomes independent.
 August 17  1962: Peter Fechter becomes the first person to be shot dead at the Berlin Wall.
 August 19  43 BC: Augustus becomes Roman consul.
 August 19  14: Augustus dies.
 August 19  1919: Afghanistan becomes independent.
 August 19  1991: The August Coup against Mikhail Gorbachev, in the Soviet Union, begins.
 August 20  1940: Leon Trotsky is fatally wounded with an ice pick in Mexico.
 August 20  1968: The Prague Spr

### Build the Prompt

In [25]:
contexts = [r["metadata"]["text"] for r in res["matches"]]
prompt_start = (
    "Answer the question based on the context below.\n\n"+
    "Context:\n"
)
prompt_end = (
    f"\n\nQuestion: write an article titled: {query}\nAnswer:"
)
prompt = (
    prompt_start + "\n\n---\n\n".join(contexts) + 
    prompt_end
)

print(prompt)

Answer the question based on the context below.

Context:
August 13  1961: Building of the Berlin Wall begins.
 August 14  1945: Japan announces its surrender at the end of World War II.
 August 14/15  1947: India is partitioned at independence from the UK, as the new mainly Islamic state of Pakistan is created.
 August 15  1960: The Republic of the Congo becomes independent.
 August 15  1971: Bahrain becomes independent.
 August 16  1977: Elvis Presley dies aged 42, leading to a worldwide outpouring of grief.
 August 17  1945: Indonesia declares independence from the Netherlands.
 August 17  1960: Gabon becomes independent.
 August 17  1962: Peter Fechter becomes the first person to be shot dead at the Berlin Wall.
 August 19  43 BC: Augustus becomes Roman consul.
 August 19  14: Augustus dies.
 August 19  1919: Afghanistan becomes independent.
 August 19  1991: The August Coup against Mikhail Gorbachev, in the Soviet Union, begins.
 August 20  1940: Leon Trotsky is fatally wounded wi

### Get the Summary

In [26]:
res = openai_client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    temperature=0,
    max_tokens=636,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)

print("-" * 80)
print(res.choices[0].text)

--------------------------------------------------------------------------------
 The Berlin Wall was a physical barrier that divided the city of Berlin, Germany from 1961 to 1989. It was built by the German Democratic Republic (GDR), also known as East Germany, in an effort to prevent its citizens from fleeing to the democratic West. The wall was made up of concrete segments, barbed wire, and guard towers, stretching 96 miles and cutting through the heart of the city. It was heavily guarded and anyone caught trying to cross it risked being shot by border guards.

The construction of the Berlin Wall began on August 13, 1961, and it quickly became a symbol of the Cold War and the division between the communist East and the capitalist West. Families and friends were separated, and the wall became a symbol of oppression and the lack of freedom in East Germany.

Despite its purpose of keeping people in, the Berlin Wall also became a symbol of hope and resistance. Many attempts were made to